### Market data refresh 

### Input Description

RAW OHLC data.

### Output  

Clean OHLC data in a hdf store

### Operations

This code takes a financial market data file and runs it through a processing pipeline. The following operations are carried out :

- Localise the time data to market time
- Merge with existing RAW data based on datetime
- Save the resulting RAW data to HDF5

In [1]:
#!pip install --upgrade "../../quantutils"
!pip install "../../marketinsights-price-aggregator"
import json, os, pandas
import quantutils.dataset.pipeline as ppl
import quantutils.core.options as opt_utils
from quantutils.api.datasource import MarketDataStore
import MIPriceAggregator.connectors as connectors
import pandas as pd
import numpy as np
import time
from datetime import datetime, date, timedelta

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
from dash import html
import plotly.express as px 

import warnings
from tables import NaturalNameWarning
warnings.filterwarnings('ignore', category=NaturalNameWarning)
from tqdm import tqdm

Processing /home/cwilkin/Development/repos/marketinsights-price-aggregator
  Preparing metadata (setup.py) ... done
  Created wheel for MIPriceAggregator: filename=MIPriceAggregator-1.0.0-py3-none-any.whl size=9470 sha256=428f9f3ac3f20d00ee49d310a8c334a8d33f78e41ebc74707d94edf8a6697061
  Stored in directory: /home/cwilkin/.cache/pip/wheels/a3/66/dc/3a75635dd1cbb1bf931d0df38e458b1890be1666bbb5b88c7f
Successfully built MIPriceAggregator
  Attempting uninstall: MIPriceAggregator
    Found existing installation: MIPriceAggregator 1.0.0
    Uninstalling MIPriceAggregator-1.0.0:
      Successfully uninstalled MIPriceAggregator-1.0.0


/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  from ipykernel import kernelapp as app


In [29]:
def getConnector(connClass, connName, tz, options):
    connectorClass = getattr(connectors, connClass)
    connectorInstance = connectorClass(connName, tz, options)
    return connectorInstance

def fetchHistoricalData(mds, ds_file, start="1979-01-01", end="2050-01-01", records=200, delta=False, newOnly=False, debug=False):

    datasources = json.load(open(ds_file))
    
    data = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'ID']))
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["ID"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:

            for source in tqdm(market["sources"]):
                
                #TODO Implement newOnly
                
                if delta:
                    try:
                        start = mds.aggregate(market["ID"], [source["ID"]]).index.get_level_values("Date_Time")[-1]
                        print(start)
                        records = (datetime.utcnow() - start.to_pydatetime().replace(tzinfo=None)).days + 1
                        print(records)
                        start = start.strftime('%Y-%m-%d')
                    except Exception as e:
                        print(e)
                        print("Could not find " + market["ID"])
                        start = "1979-01-01"
                    
                newData = dataConnector.getData(market, source, start, end, records)

                if newData is not None:

                    if debug:
                        print("Adding " + source["ID"] + " to " + market["ID"] + " table")
                        print(newData)

                    if mds is not None:  
                        mds.append(market["ID"], newData, update=True)

                    data = ppl.merge(data, newData)

    return data

                
def fetchHistoricalOptionData(mds, ds_file, start="1979-01-01", end="2050-01-01", records=200, refreshUnderyling=False, dry_run=False, debug=False ):
    
    datasources = json.load(open(ds_file))
    
    data = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'ID']))
    
    #First ensure that all underlying date is updated
    if refreshUnderyling:
        fetchHistoricalData(mds, ds_file, start=start, end=end, records=records, debug=debug)
        
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["ID"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:
            
            dataConnector.setState({"marketData": mds.get(market["ID"])})
            
            for optionChain in market["optionChains"]:                
                    
                # TODO: Implement newOnly

                newData = dataConnector.getOptionData(optionChain, start, end, records, debug)

                if newData is not None:

                    print("Adding " + optionChain["ID"] + " to " + market["ID"] + " table")

                    if debug:
                        print(newData)

                    if not dry_run:  
                        mds.append(market["ID"] + "_Options", newData, update=True)

                    data = ppl.merge(data, newData)
    return data
                   
    
def getOptionChains(datasource, root):
    
    dataConnector = getConnector(datasource["class"], datasource["ID"], datasource["timezone"], datasource["opts"])
    
    chains = dataConnector.getOptionChains(root)
    
    # Update expiry dates
    dates = []
    for _,chain in chains.iterrows():
        if np.isnan(chain["expiry"]):
            info = dataConnector.getOptionInfo(chain["ID"])
            dates.append(info["contractExpirationDate"])
        else:
            dates.append(str(date.today() + timedelta(int(chain["expiry"]) + 1)))
    chains["expiry"] = dates
    
    return chains


In [3]:
# Local Options
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
#mds = MarketDataStore(location="../datastore")
#options = fetchHistoricalOptionData(mds, "../datasources/datasources_BarChartOption.json", records=200, refreshUnderyling=False, debug=True)
options = fetchHistoricalOptionData(mds, "../datasources/datasources_MDS.json", dry_run=True)
#options = fetchHistoricalOptionData(mds, "../datasources/datasources_BarChartOption.json", records=1)
options

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/MIPriceAggregator/connectors/MDSConnector.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/MIPriceAggregator/connectors/MDSConnector.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/MIPriceAggregator/connectors/MDSConnector.py:41: SettingWithCopyWarning:


A value is trying to be set on 

Adding CLV22 to WTICrudeOil table
Adding CLY00 to WTICrudeOil table
Adding CLX22 to WTICrudeOil table
Adding CLZ22 to WTICrudeOil table
Adding CLF23 to WTICrudeOil table
Adding CLG23 to WTICrudeOil table
Adding CLH23 to WTICrudeOil table
Adding CLJ23 to WTICrudeOil table
Adding CLK23 to WTICrudeOil table
Adding CLM23 to WTICrudeOil table
Adding CLN23 to WTICrudeOil table
Adding CLQ23 to WTICrudeOil table
Adding CLU23 to WTICrudeOil table
Adding CLV23 to WTICrudeOil table
Adding CLX23 to WTICrudeOil table
Adding CLZ23 to WTICrudeOil table
Adding CLF24 to WTICrudeOil table
Adding CLG24 to WTICrudeOil table
Adding CLH24 to WTICrudeOil table
Adding CLJ24 to WTICrudeOil table
Adding CLK24 to WTICrudeOil table


Close   High    Low  Open  OpenInterest  \
Date_Time                 ID                                                   
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23   6.23  6.23         100.0   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13   6.13  6.13         100.0   
2020-01-01 00:00:00+00:00 CLZ2|540C    NaN    NaN    NaN   NaN           0.0   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35   6.35  6.35         100.0   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21   6.21  6.21         100.0   
...                                    ...    ...    ...   ...           ...   
2022-10-22 00:00:00+00:00 CLZ3|940C   6.83   6.83   6.83  0.00           1.0   
                          CLZ3|945C   6.70   6.70   6.70  0.00          82.0   
                          CLZ3|950C   6.58   6.58   6.58  0.00        5254.0   
                          CLZ3|950P  25.93  25.93  25.93  0.00           5.0   
                          CLZ3|990C   5.70   5.70   5.70  0.00           1.0   

                                     Volume                    instrumentName  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2019-12-31 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-01 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-02 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-03 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
...                                     ...                               ...   
2022-10-22 00:00:00+00:00 CLZ3|940C     0.0  Crude Oil WTI Dec '23 94.00 Call   
                          CLZ3|945C     0.0  Crude Oil WTI Dec '23 94.50 Call   
                          CLZ3|950C     6.0  Crude Oil WTI Dec '23 95.00 Call   
                          CLZ3|950P     0.0   Crude Oil WTI Dec '23 95.00 Put   
                          CLZ3|990C     0.0  Crude Oil WTI Dec '23 99.00 Call   

                                     strike type  underlying underlyingSymbol  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C    54.0    c       52.01            CLZ22   
2019-12-31 00:00:00+00:00 CLZ2|540C    54.0    c       51.58            CLZ22   
2020-01-01 00:00:00+00:00 CLZ2|540C    54.0    c         NaN            CLZ22   
2020-01-02 00:00:00+00:00 CLZ2|540C    54.0    c       51.97            CLZ22   
2020-01-03 00:00:00+00:00 CLZ2|540C    54.0    c       51.72            CLZ22   
...                                     ...  ...         ...              ...   
2022-10-22 00:00:00+00:00 CLZ3|940C    94.0    c         NaN            CLZ23   
                          CLZ3|945C    94.5    c         NaN            CLZ23   
                          CLZ3|950C    95.0    c         NaN            CLZ23   
                          CLZ3|950P    95.0    p         NaN            CLZ23   
                          CLZ3|990C    99.0    c         NaN            CLZ23   

                                       ask    bid                    expiry  \
Date_Time                 ID                                                  
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23 2022-11-21 00:00:00+00:00   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13 2022-11-21 00:00:00+00:00   
2020-01-01 00:00:00+00:00 CLZ2|540C    NaN    NaN 2022-11-21 00:00:00+00:00   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35 2022-11-21 00:00:00+00:00   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21 2022-11-21 00:00:00+00:00   
...                                    ...    ...                       ...   
2022-10-22 00:00:00+00:00 CLZ3|940C   6.83   6.83 2023-11-20 00:00:00+00:00   
                          CLZ3|945C   6.70   6.70 2023-11-20 00:00:00+00:00   
                          CLZ3|950C   6.58   6.58 2023-11-20 00:00:00+00:00   
                   

In [4]:
options.dropna()

Close   High    Low  Open  OpenInterest  \
Date_Time                 ID                                                   
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23   6.23  6.23         100.0   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13   6.13  6.13         100.0   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35   6.35  6.35         100.0   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21   6.21  6.21         100.0   
2020-01-06 00:00:00+00:00 CLZ2|540C   6.57   6.57   6.57  6.57         100.0   
...                                    ...    ...    ...   ...           ...   
2022-10-21 00:00:00+00:00 CLZ3|940C   6.83   6.85   6.81  0.00           1.0   
                          CLZ3|945C   6.70   6.73   6.69  0.00          82.0   
                          CLZ3|950C   6.58   6.61   6.57  0.00        5254.0   
                          CLZ3|950P  25.93  26.19  26.15  0.00           5.0   
                          CLZ3|990C   5.70   5.72   5.69  0.00           1.0   

                                     Volume                    instrumentName  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2019-12-31 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-02 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-03 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-06 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
...                                     ...                               ...   
2022-10-21 00:00:00+00:00 CLZ3|940C     0.0  Crude Oil WTI Dec '23 94.00 Call   
                          CLZ3|945C     0.0  Crude Oil WTI Dec '23 94.50 Call   
                          CLZ3|950C     0.0  Crude Oil WTI Dec '23 95.00 Call   
                          CLZ3|950P     0.0   Crude Oil WTI Dec '23 95.00 Put   
                          CLZ3|990C     0.0  Crude Oil WTI Dec '23 99.00 Call   

                                     strike type  underlying underlyingSymbol  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C    54.0    c       52.01            CLZ22   
2019-12-31 00:00:00+00:00 CLZ2|540C    54.0    c       51.58            CLZ22   
2020-01-02 00:00:00+00:00 CLZ2|540C    54.0    c       51.97            CLZ22   
2020-01-03 00:00:00+00:00 CLZ2|540C    54.0    c       51.72            CLZ22   
2020-01-06 00:00:00+00:00 CLZ2|540C    54.0    c       51.81            CLZ22   
...                                     ...  ...         ...              ...   
2022-10-21 00:00:00+00:00 CLZ3|940C    94.0    c       75.04            CLZ23   
                          CLZ3|945C    94.5    c       75.04            CLZ23   
                          CLZ3|950C    95.0    c       75.04            CLZ23   
                          CLZ3|950P    95.0    p       75.04            CLZ23   
                          CLZ3|990C    99.0    c       75.04            CLZ23   

                                       ask    bid                    expiry  \
Date_Time                 ID                                                  
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23 2022-11-21 00:00:00+00:00   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13 2022-11-21 00:00:00+00:00   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35 2022-11-21 00:00:00+00:00   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21 2022-11-21 00:00:00+00:00   
2020-01-06 00:00:00+00:00 CLZ2|540C   6.57   6.57 2022-11-21 00:00:00+00:00   
...                                    ...    ...                       ...   
2022-10-21 00:00:00+00:00 CLZ3|940C   6.83   6.83 2023-11-20 00:00:00+00:00   
                          CLZ3|945C   6.70   6.70 2023-11-20 00:00:00+00:00   
                          CLZ3|950C   6.58   6.58 2023-11-20 00:00:00+00:00   
                   

In [10]:
mds.get("WTICrudeOil_Options")

Close   High    Low  Open  OpenInterest  \
Date_Time                 ID                                                   
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23   6.23  6.23         100.0   
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13   6.13  6.13         100.0   
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35   6.35  6.35         100.0   
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21   6.21  6.21         100.0   
2020-01-06 00:00:00+00:00 CLZ2|540C   6.57   6.57   6.57  6.57         100.0   
...                                    ...    ...    ...   ...           ...   
2022-10-21 00:00:00+00:00 CLZ3|940C   6.83   6.85   6.81  0.00           1.0   
                          CLZ3|945C   6.70   6.73   6.69  0.00          82.0   
                          CLZ3|950C   6.58   6.61   6.57  0.00        5254.0   
                          CLZ3|950P  25.93  26.19  26.15  0.00           5.0   
                          CLZ3|990C   5.70   5.72   5.69  0.00           1.0   

                                     Volume                    instrumentName  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2019-12-31 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-02 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-03 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
2020-01-06 00:00:00+00:00 CLZ2|540C     0.0  Crude Oil WTI Dec '22 54.00 Call   
...                                     ...                               ...   
2022-10-21 00:00:00+00:00 CLZ3|940C     0.0  Crude Oil WTI Dec '23 94.00 Call   
                          CLZ3|945C     0.0  Crude Oil WTI Dec '23 94.50 Call   
                          CLZ3|950C     0.0  Crude Oil WTI Dec '23 95.00 Call   
                          CLZ3|950P     0.0   Crude Oil WTI Dec '23 95.00 Put   
                          CLZ3|990C     0.0  Crude Oil WTI Dec '23 99.00 Call   

                                     strike type  underlying underlyingSymbol  \
Date_Time                 ID                                                    
2019-12-30 00:00:00+00:00 CLZ2|540C    54.0    c       52.01            CLZ22   
2019-12-31 00:00:00+00:00 CLZ2|540C    54.0    c       51.58            CLZ22   
2020-01-02 00:00:00+00:00 CLZ2|540C    54.0    c       51.97            CLZ22   
2020-01-03 00:00:00+00:00 CLZ2|540C    54.0    c       51.72            CLZ22   
2020-01-06 00:00:00+00:00 CLZ2|540C    54.0    c       51.81            CLZ22   
...                                     ...  ...         ...              ...   
2022-10-21 00:00:00+00:00 CLZ3|940C    94.0    c       75.04            CLZ23   
                          CLZ3|945C    94.5    c       75.04            CLZ23   
                          CLZ3|950C    95.0    c       75.04            CLZ23   
                          CLZ3|950P    95.0    p       75.04            CLZ23   
                          CLZ3|990C    99.0    c       75.04            CLZ23   

                                       ask    bid                    expiry  
Date_Time                 ID                                                 
2019-12-30 00:00:00+00:00 CLZ2|540C   6.23   6.23  2022-11-21T00:00:00.000Z  
2019-12-31 00:00:00+00:00 CLZ2|540C   6.13   6.13  2022-11-21T00:00:00.000Z  
2020-01-02 00:00:00+00:00 CLZ2|540C   6.35   6.35  2022-11-21T00:00:00.000Z  
2020-01-03 00:00:00+00:00 CLZ2|540C   6.21   6.21  2022-11-21T00:00:00.000Z  
2020-01-06 00:00:00+00:00 CLZ2|540C   6.57   6.57  2022-11-21T00:00:00.000Z  
...                                    ...    ...                       ...  
2022-10-21 00:00:00+00:00 CLZ3|940C   6.83   6.83  2023-11-20T00:00:00.000Z  
                          CLZ3|945C   6.70   6.70  2023-11-20T00:00:00.000Z  
                          CLZ3|950C   6.58   6.58  2023-11-20T00:00:00.000Z  
                          CLZ3

In [ ]:
# Update options config file with all option chains
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))
market = datasources[0]["markets"][0]

# Get Chains
chains = getOptionChains(datasources[0], market["optionRoot"])
market["optionChains"].extend(json.loads(chains.to_json(orient="records")))

# Get Options and Sources from Chains
options, datasources = fetchOptionData(datasources)

# Add underlyings to sources
for optionChain in market["optionChains"]:
    market["sources"].extend([{"ID": optionChain["underlying"], "sample_unit": "D"}])

#with open(ds_file, 'w', encoding='utf-8') as f:
#    json.dump(datasources, f, ensure_ascii=False, indent=4)
    

In [12]:
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))

mkt = datasources[0]["markets"][0]
srcs = []
options = []
for source in mkt["sources"]:
    srcs.append({"ID": source["ID"], "sample_unit": source["sample_unit"]})
    strike = float(source["ID"][5:-1])
    type = source["ID"][-1].lower()
    if type == 'c':
        typeString = "Call"
    else:
        typeString = "Put"
    options.append({
        "ID": source["ID"],
        "instrumentName": "Crude Oil WTI Oct '22 {} {}".format(strike, typeString),
        "strike": strike,
        "type": type
    })

mkt["sources"] = srcs
mkt["optionChains"][0]["options"] = options

#with open(ds_file, 'w', encoding='utf-8') as f:
#    json.dump(datasources, f, ensure_ascii=False, indent=4)

In [ ]:
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
#data = mds.aggregate("WTICrudeOil", ["CLV22"])
clf = mds.aggregate("WTICrudeOil", ["CLV22"])

In [36]:
import plotly.express as px
df = px.data.gapminder()

In [ ]:
# Local Options
mds = MarketDataStore(location="../datastore")
options = fetchHistoricalData(mds, "../datasources/datasources_BarChartOption.json", records=300, newOnly=True)

In [133]:
# Remote (cluster)
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
clf = mds.aggregate("WTICrudeOil", ["CLV22"])

In [11]:
# Remote (localhost)
mds = MarketDataStore(remote=True, location="http://localhost:8080")
refreshMarketData(mds, "../datasources", "datasources.json")

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /prices/aggregate/WTICrudeOil?start=1979-01-01&end=2050-01-01&sources=CLV22 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9618b262b0>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [8]:
df1 = mds.aggregate(["D&J-IND","SANDP-500"], "H", "1979-01-01", "2050-01-01", debug=True)
df1

Loading data from D&J-IND in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods
Loading data from SANDP-500 in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods


,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.480000,13374.820000,13338.680000,13345.450000
2013-01-02 16:00:00+00:00,13345.600000,13353.750000,13321.640000,13321.770000
2013-01-02 17:00:00+00:00,13321.760000,13339.250000,13320.280000,13326.660000
2013-01-02 18:00:00+00:00,13326.640000,13336.060000,13322.510000,13329.190000
2013-01-02 19:00:00+00:00,13329.090000,13339.820000,13318.190000,13331.800000
...,...,...,...,...
2018-03-19 16:00:00+00:00,24632.111675,24650.292289,24554.939070,24583.990050
2018-03-19 17:00:00+00:00,24579.349894,24585.490101,24473.096306,24568.679534
2018-03-19 18:00:00+00:00,24570.309589,24590.980286,24455.305705,24533.498346


In [5]:
clf=clf.reset_index().set_index("Date_Time")
abc = (np.log(clf.Close) - np.log(clf.Close.shift(1))).rolling(20).std()*252**.5
abc

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Date_Time
2020-01-02 00:00:00+00:00         NaN
2020-01-03 00:00:00+00:00         NaN
2020-01-06 00:00:00+00:00         NaN
2020-01-07 00:00:00+00:00         NaN
2020-01-08 00:00:00+00:00         NaN
                               ...   
2022-10-07 00:00:00+00:00    0.456785
2022-10-10 00:00:00+00:00    0.460890
2022-10-11 00:00:00+00:00    0.472405
2022-10-12 00:00:00+00:00    0.474098
2022-10-14 00:00:00+00:00    0.457241
Name: Close, Length: 704, dtype: float64

In [11]:
import plotly.express as px 
px.line(x=abc.index, y=abc)

In [50]:
px.line(x=clf.index, y=clf.Close,labels={"y": "Price"}, title="Historical prices for CL")

In [17]:
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
mds.append("WTICrudeOil_Options")

{'rc': 'success'}

In [25]:
optionsData['expiry']

0         2022-11-21T00:00:00.000Z
1         2022-11-21T00:00:00.000Z
2         2022-11-21T00:00:00.000Z
3         2022-11-21T00:00:00.000Z
4         2022-11-21T00:00:00.000Z
                    ...           
373428    2023-11-20T00:00:00.000Z
373429    2023-11-20T00:00:00.000Z
373430    2023-11-20T00:00:00.000Z
373431    2023-11-20T00:00:00.000Z
373432    2023-11-20T00:00:00.000Z
Name: expiry, Length: 373433, dtype: object

In [15]:
optionsData = options.reset_index()
#optionsData["timeToExpiry"] = [((exp.days * 24. * 3600. + exp.seconds + 3600) / (24. * 3600.)) for exp in (pd.to_datetime(optionsData['expiry']).sub(optionsData['Date_Time']))]
optionsData["IV"] = opt_utils.get_IV(optionsData, 0.0).values

/home/cwilkin/Development/anaconda/envs/dev/lib/python3.6/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning:

Found Below Intrinsic contracts at index [294538, 296738, 301194, 317105, 375190, 377756, 398855, 398857, 398859, 398861, 398863, 398865, 398867, 398869, 398871, 411604, 411606, 411617, 422550, 422552, 423332, 433397, 433399, 433401, 433404, 433406, 433409, 433412, 433415, 433420, 433423, 433425, 433428, 433433, 433436, 433438, 436690, 439196, 439198, 439506, 442022, 442024, 442035, 442040, 442042, 442044, 442046, 453390, 453395, 453397, 453399, 453401, 453404, 453406, 453409, 453411, 453413, 453415, 453417, 453419, 453421, 453423, 453425, 453427, 453429, 453431, 453433, 453435, 456245, 456247, 456258, 456263, 456265, 456267, 456269, 456272, 456274, 456277, 456279, 456281, 456283, 456285, 456287, 456289, 456291, 456293, 456295, 456297, 456299, 456301, 456303, 456305, 456307, 456309, 456311, 456313, 456315, 456317, 456319, 456321, 456323, 456325, 458879, 4

In [25]:
data= mds.get("WTICrudeOil_Options")

In [28]:
optionsData = data.reset_index()
puts = optionsData[optionsData["type"]=="p"].dropna()

#p = puts[puts["underlyingSymbol"]=="CLZ22"].sort_values("timeToExpiry")
p=puts[puts["Date_Time"]=="2022-10-14"].sort_values("timeToExpiry")

import plotly.express as px
px.scatter(p, x="strike", y="IV", animation_frame="timeToExpiry")

In [13]:
optionData = optionData.reset_index()

od = optionData.set_index(["underlyingSymbol", "Date_Time"])
od = od[od["type"]=="p"]

a = []
for symbol, df in od.groupby(level=0):
    x = pd.DataFrame()
    for date, df2 in df.groupby(level=1):
        strike = (min(df2["strike"], key=lambda x:abs(x-df2["underlying"][0])))
        x = pd.concat([x, df2[df2["strike"]==strike]])
    a.append(x)
    
IV = pd.DataFrame()
for x in a:
    opts = opt_utils.get_IV(x.reset_index())
    IV = pd.concat([IV, opts[["Date_Time", "impl"]].set_index("Date_Time").rename(columns={"impl":str(opts["expiry"][0])})], axis=1)
    
px.line(IV, x=IV.index, y=IV.columns)

#a = IV.copy()
#a[(~((IV.values > pd.concat([IV.mean(axis=1) + (2*IV.std(axis=1))] * 16, axis=1, ignore_index=True)) | (IV.values < pd.concat([IV.mean(axis=1) - (2*IV.std(axis=1))] * 16, axis=1, ignore_index=True)))).values]=np.nan
#a["mean"] = IV.mean(axis=1)

NameError: name 'optionData' is not defined

In [24]:
c["mean"] = IV.mean(axis=1)
abc

Date_Time
2020-01-02 00:00:00+00:00                                                  NaN
2020-01-03 00:00:00+00:00                                                  NaN
2020-01-06 00:00:00+00:00                                                  NaN
2020-01-07 00:00:00+00:00                                                  NaN
2020-01-08 00:00:00+00:00                                                  NaN
                                                   ...                        
2022-10-10 00:00:00+00:00                                              0.46089
2022-10-11 00:00:00+00:00                                             0.472405
2022-10-12 00:00:00+00:00                                             0.474098
2022-10-14 00:00:00+00:00                                             0.457241
mean                         Date_Time
2020-01-02 00:00:00+00:00     97423....
Name: Close, Length: 705, dtype: object

In [53]:
m = IV.mean()
c = pd.concat([IV,abc])
c["mean"] = IV.mean(axis=1)
fig = px.line(c, x=c.index, y=[0,"2022-12-16 00:00:00"])


In [19]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [30]:
IV

,2022-12-16 00:00:00,2023-12-15 00:00:00,2023-01-18 00:00:00,2023-02-16 00:00:00,2024-02-15 00:00:00,2023-03-16 00:00:00,2024-03-15 00:00:00,2023-04-17 00:00:00,2023-05-17 00:00:00,2023-06-15 00:00:00,2023-07-17 00:00:00,2023-08-17 00:00:00,2023-09-15 00:00:00,2022-10-17 00:00:00,2022-11-21 00:00:00,2023-11-20 00:00:00
Date_Time,,,,,,,,,,,,,,,,
2021-07-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296700,NaN
2021-07-28 00:00:00+00:00,NaN,NaN,NaN,0.321004,NaN,NaN,NaN,NaN,0.312761,NaN,NaN,NaN,NaN,NaN,0.300531,0.300932
2021-07-29 00:00:00+00:00,NaN,NaN,NaN,0.315610,NaN,NaN,NaN,NaN,0.307901,NaN,NaN,NaN,NaN,NaN,0.286025,0.296757
2021-07-30 00:00:00+00:00,NaN,NaN,NaN,0.317486,NaN,NaN,NaN,NaN,0.309509,NaN,NaN,NaN,NaN,NaN,0.286185,0.297146
2021-08-02 00:00:00+00:00,NaN,NaN,NaN,0.321301,NaN,NaN,NaN,NaN,0.313848,NaN,NaN,NaN,NaN,NaN,0.300366,0.301062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-11 00:00:00+00:00,0.516901,0.470268,0.519666,0.525348,0.466482,0.528290,0.518374,0.523998,0.520762,0.506850,0.519314,0.512719,0.502746,0.471290,0.476101,0.480747
2022-10-12 00:00:00+00:00,0.511873,0.462633,0.515709,0.523157,0.460255,0.529507,0.515846,0.523385,0.521482,0.522306,0.514264,0.506927,0.496347,0.458934,0.472001,0.475284
2022-10-13 00:00:00+00:00,0.503895,0.464269,0.509605,0.517321,0.461689,0.522325,0.514188,0.519249,0.516072,0.502525,0.513458,0.505799,0.495872,0.403988,0.459127,0.474580


In [46]:
from dash_html_template import Template
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

app = JupyterDash(external_stylesheets=[dbc.themes.DARKLY, dbc.icons.BOOTSTRAP])
load_figure_template(["bootstrap", "materia", "darkly"])

fig = px.line(x=abc.index, y=abc, template="darkly")

html_layout = """
    <div>
        <nav class="navbar navbar-inverse navbar-dark bg-primary navbar-expand-sm navbar-fixed-top">
          <div class="container-fluid">
            <div class="navbar-header">
              <a class="navbar-brand" href="">Squirrel Analytics</a>
            </div>
            <div id="navbar" class="navbar-header">
              <ul class="nav navbar-nav navbar-right">
                <li><a class="navbar-brand" href="https://github.ibm.com/apimesh/dashapp/wiki" target="_blank">Help</a></li>
                <li><a class="navbar-brand" href="https://github.ibm.com/apimesh/dashapp/issues" target="_blank">Support</a></li>
                <li><a class="navbar-brand" href="">wilkobets <i class="bi-person"></a></i></li>   
              </ul>
            </div>
          </div>
        </nav>
        <div>
          <h1>Avocado Analytics</h1>
          <p>
            Analyze the behavior of avocado prices and the number
            of avocados sold in the US between 2015 and 2018
          </p>
          <!-- Rest of the app -->
          <template id='graph'></template>
        </div>
    </div>
    """
# <template id='graph'></template>
    
injection_dict = {'graph': dcc.Graph(id="life-exp-vs-gdp", figure=None)}

#app.layout = html.Div([dcc.Graph(id="life-exp-vs-gdp", figure=fig)])
app.layout = Template.from_string(html_layout, injection_dict)

app.run_server(host='0.0.0.0',debug=True)

Dash app running on http://0.0.0.0:8050/


In [4]:
'{:40.40}'.format(data["instrumentName"])

NameError: name 'data' is not defined

In [36]:
#clf = clf[clf["ID"]=="CLV22"]
data["poo"] = data.join(clf,lsuffix='_caller', on="Date_Time")["Close"]
#underlyingVals = clf.loc[data.dropna()["Date_Time"], "Close"]
#optionData["underlying"] = np.nan
#optionData.loc[underlyingVals.index.get_level_values(0), "underlying"] = underlyingVals.values


In [160]:
x = np.array(len(opt.index) * [np.nan])

x[opt.index.get_level_values(0).isin(underlyingVals.index.get_level_values(0))] = underlyingVals
x

array([ 72.21,  72.56,  70.97,    nan,    nan,  72.23,  73.23,  73.44,
        74.24,  73.9 ,    nan,    nan,  73.23,  75.61,  76.33,  76.18,
        77.35,    nan,    nan,    nan,  78.55,  79.17,  79.52,  78.91,
          nan,    nan,  77.45,  78.75,  79.93,  79.29,  79.23,    nan,
          nan,  79.83,  79.71,  80.14,  81.5 ,  82.8 ,    nan,    nan,
        82.71,  81.18,  82.15,  81.99,  83.52,    nan,    nan,  84.43,
        81.5 ,  82.45,  81.49,  82.25,    nan,    nan,    nan,  83.72,
        84.38,  83.3 ,  82.63,    nan,    nan,  84.08,  85.89,  89.59,
        89.75,  96.79,    nan,    nan,  99.45, 100.77,  89.14,  89.61,
        93.26,    nan,    nan,  89.41,  86.1 ,  84.68,  90.65,  91.91,
          nan,    nan,  96.33,  95.9 ,  99.52,  97.19,  98.66,    nan,
          nan,  92.82,  92.65,  96.08,  91.82,  93.17,    nan,    nan,
        96.66,  95.88,  91.98,  92.68,  94.66,    nan,    nan,  91.86,
        96.57,  99.32, 100.97,    nan,    nan,    nan, 101.38,  96.8 ,
      

In [162]:
opt

,,Close,High,Low,Open,OpenInterest,Volume
Date_Time,ID,,,,,,
2021-12-29 00:00:00+00:00,CLV2|500P,2.68,2.68,2.68,2.68,3.0,3.0
2021-12-30 00:00:00+00:00,CLV2|500P,2.64,2.64,2.64,2.64,4.0,1.0
2021-12-31 00:00:00+00:00,CLV2|500P,2.87,2.82,2.87,2.87,4.0,0.0
2022-01-01 00:00:00+00:00,CLV2|500P,NaN,NaN,NaN,NaN,0.0,0.0
2022-01-02 00:00:00+00:00,CLV2|500P,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...
2022-09-10 00:00:00+00:00,CLV2|500P,NaN,NaN,NaN,NaN,0.0,0.0
2022-09-11 00:00:00+00:00,CLV2|500P,NaN,NaN,NaN,NaN,0.0,0.0
2022-09-12 00:00:00+00:00,CLV2|500P,0.01,0.01,0.01,0.01,4509.0,0.0
